In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [2]:
# Đọc dữ liệu
matches = pd.read_csv("matches.csv", index_col=0)

In [ ]:
# Tiền xử lý dữ liệu
# Chuyển đổi cột "date" từ chuỗi thành kiểu datetime
matches["date"] = pd.to_datetime(matches["date"])
# Tạo cột "target" (nhãn) cho kết quả trận đấu, với giá trị 1 nếu đội thắng (kết quả là "W"), ngược lại là 0
matches["target"] = (matches["result"] == "W").astype("int")
# Mã hóa các giá trị trong cột "venue" (sân đấu) thành các mã số bằng phương pháp category codes
matches["venue_code"] = matches["venue"].astype("category").cat.codes
# Mã hóa các giá trị trong cột "opponent" (đối thủ) thành các mã số bằng phương pháp category codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
# Tạo cột "hour" từ cột "time" bằng cách tách ra chỉ phần giờ (trước dấu hai chấm)
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
# Tạo cột "day_code" từ cột "date" chứa mã số của ngày trong tuần (0 là Thứ Hai, 6 là Chủ Nhật)
matches["day_code"] = matches["date"].dt.dayofweek

In [4]:
# Định nghĩa các đặc trưng
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [5]:
# Hàm tính toán rolling averages
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [6]:
# Các cột cần tính rolling averages
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [7]:
# Áp dụng rolling averages cho tất cả các đội
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

C:\Users\Kaka Tran\AppData\Local\Temp\ipykernel_14648\464013960.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [8]:
# Hàm dự đoán
def make_predictions(data, predictors):
    train = data[data["date"] < '2021-01-01']
    test = data[(data["date"] >= '2021-01-01') & (data["date"] < '2022-01-01')]
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [ ]:
# Huấn luyện mô hình và dự đoán cho năm 2021 sử dụng dữ liệu năm 2020
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
combined, prediction_2021 = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
# Inho dự đoán năm ra độ chính xác c 2021
print(f"Độ chính xác cho dự đoán năm 2021: {prediction_2021}")

Độ chính xác cho dự đoán năm 2021: 0.4864864864864865


In [ ]:
# Kiểm tra kết quả thực tế và dự đoán cho năm 2021
print(combined.head())

    actual  predicted
13       1          1
14       0          0
15       1          0
16       1          0
17       0          0


In [12]:
# Hàm dự đoán cho năm 2022
def make_predictions_for_2022(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] >= '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error


In [ ]:
# Huấn luyện mô hình và dự đoán cho năm 2022 sử dụng dữ liệu năm 2021
combined_2022, prediction_2022 = make_predictions_for_2022(matches_rolling, predictors + new_cols)

In [ ]:
# In ra độ chính xác cho dự đoán năm 2022
print(f"Độ chính xác cho dự đoán năm 2022: {prediction_2022}")

Độ chính xác cho dự đoán năm 2022: 0.625


In [19]:
# Kiểm tra kết quả thực tế và dự đoán cho năm 2022
print(combined_2022.head(100))

     actual  predicted
54        0          0
55        0          0
56        1          0
57        1          0
58        1          1
..      ...        ...
427       1          0
428       0          0
429       1          0
430       0          0
431       1          0

[100 rows x 2 columns]
